Book Einführung in Optimirungsmodelle from Nathan Sudermann-Merx page 120

In [1]:
import polars as pl
import pyoframe as pf
from pathlib import Path

In [4]:
def read_hourly_prices():
    """Read hourly prices from CSV file and return a DataFrame.
    Special attention is paid to properly parse daylight saving time (DST) changes."""
    df = pl.read_csv(
        Path("./elspot-prices_2021_hourly_eur.csv"),
        try_parse_dates=True,
    ).drop_nulls(subset=["DE-LU"])

    df = df.select(
        pl.datetime(
            pl.col("Date").dt.year(),
            pl.col("Date").dt.month(),
            pl.col("Date").dt.day(),
            pl.col("Hours").str.slice(0, 2).cast(pl.Int32),
            time_zone="Europe/Prague",
            ambiguous=pl.when(
                pl.concat_str(pl.col("Date"), pl.col("Hours")).is_first_distinct()
            )
            .then(pl.lit("earliest"))
            .otherwise(pl.lit("latest")),
        ).alias("tick"),
        pl.col("DE-LU").str.replace(",", ".", literal=True).cast(float).alias("price"),
    )

    return df

In [5]:
hourly_prices = read_hourly_prices()
pump_max, turb_max = 70, 90
storage_min, storage_max = 100, 630
storage_level_init_and_final = 300
efficiency = 0.75

Decision variables:

In [6]:
m = pf.Model("unit commitment problem", solver='highs', use_var_names=True)

m.Pump = pf.Variable(hourly_prices[["tick"]], vtype=pf.VType.BINARY)
# ub is redundant since it will be set also in logical condition that pump and turbine cannot work at the same time
m.Turb = pf.Variable(hourly_prices[["tick"]], lb=0, ub=turb_max)
m.Storage_level = pf.Variable(
    hourly_prices[["tick"]], lb=storage_min, ub=storage_max
)
m.initial_storage_level = (
        m.Storage_level.filter(
            pl.col("tick") == hourly_prices["tick"].min()
        )
        == storage_level_init_and_final
)

Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms


In [7]:
m.intermediate_storage_level = (
        m.Storage_level.next(dim="tick", wrap_around=True)
        == m.Storage_level + m.Pump * pump_max * efficiency - m.Turb
)

m.pump_and_turbine_xor = m.Turb <= (1 - m.Pump) * turb_max

m.maximize = pf.sum((m.Turb - pump_max * m.Pump) * hourly_prices)

m.attr.RelativeGap = 1e-5

m.optimize()

Hessian has dimension 26281 but no nonzeros, so is ignored
MIP  has 17521 rows; 26281 cols; 61322 nonzeros; 8760 integer variables (8760 binary)
Coefficient ranges:
  Matrix [1e+00, 3e+02]
  Cost   [1e-02, 4e+04]
  Bound  [1e+00, 6e+02]
  RHS    [0e+00, 0e+00]
Presolving model
17520 rows, 26279 cols, 52558 nonzeros  0s
17518 rows, 26277 cols, 52554 nonzeros  0s

Solving MIP model with:
   17518 rows
   26277 cols (8760 binary, 0 integer, 0 implied int., 17517 continuous)
   52554 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   

In [9]:
m.write('unit_commitment_problem.lp')
m.write('unit_commitment_problem.mps')
m.Turb.solution.write_csv('turbine.csv')
m.Storage_level.solution.write_csv('storage.csv')
m.Pump.solution.write_csv('pump.csv')

Writing the model to unit_commitment_problem.lp
Writing the model to unit_commitment_problem.mps
